# PLAID to I.DOT notebook

### Input: PLAID result

 

In [139]:
#@title load modules and select input PLAID file
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import glob
import os
import math
import re
import datetime
import string
from ast import literal_eval

from google.colab import files
PLAID = files.upload()

Saving PLAID_RMS_SPECS.csv to PLAID_RMS_SPECS (2).csv


In [140]:
#@title User input 

# https://dispendix.com/idot-dispensing-plates/

user_name             = 'Jonne' #@param ["Jonne","Amelie","Christa","Malin","Martin","Polina","Axel","David","Ovidiu"]
protocol_name         = "RMS_SPECS" #@param {type:"string"} # Name of the protocol

sourceplate_type      = "S.100 Plate" #@param ["S.100 Plate","S.60 Plate","S.200 Plate"] 
target_plate_type     = 'MWP 384' #@param ["MWP 96","MWP 384"]

working_volume_ul     = 40 #@param {type:"number"}
V2_ul                 = working_volume_ul

x = datetime.datetime.now()

software      = "1.7.2021.1019" # I-DOT Assay Studio software version
user_name     = user_name # User name/ID
date          = (x.strftime("%x")) 
time          = (x.strftime("%X")) 


max_volume       = 8.0E-5 # Source plate max volume (80000nL = 80uL= 8.0E-5 L)
waste            = "Waste Tube" # Position of the waste well on the target carrier

dispense_to_waste          = True # Enable/disable priming before dispensing (=True/False) 
dispense_to_waste_cycles   = 3    # Number of priming cycles for each source well (=1/2/3)
dispense_to_waste_volume   = 1e-7 # Dispensing volume for each priming cycle (=5e-8/.../1e-6) 
use_deionisation           = True
optimization_level         = "ReorderAndParallel" # Used protocol optimization process to reduce total dispensing time. Possible values are NoOptimization / Reorder / ReorderAndParalell
waste_error_handling_level = "Ask" # Checkpoint for the dispensing run if no droplets are detected during priming. Possible values are Ask / Abort / Continue 
save_liquids               = "Ask" # Checkpoint for Liquid Library handling. Possible options are Ask / Never


In [141]:
#@title show compounds, doses and replicates
file = next(iter(PLAID))
df = pd.read_csv(file, dtype={'cmpdname': object} )

replicates = df[["cmpdname","CONCuM"]].value_counts().to_frame('counts')
replicates

,,counts
cmpdname,CONCuM,
DMSO,0.10%,100
DMSO-ARMS,0.10%,96
DMSO-ERMS,0.10%,64
[etop],X,5
[fenb],X,5
...,...,...
79,10,1
278,10,1
279,10,1


In [142]:
#@title Combinations

reformat_combinations         = False #@param {type:"boolean"}

if reformat_combinations == True:

  df_combinations = df.copy()
  df_combinations = df[df.cmpdname.str.match(r'[\[*,*\]]')] # subset only rows with combinations
  df_singles      = df[~df.cmpdname.str.match(r'[\[*,*\]]')] # subset rows without combinations
  print(len(df_combinations), "combinations")
  print(len(df_singles), "single compounds")

  #print(df_combinations["cmpdname"].apply(type)) # check datatype
  df_combinations["cmpdname"]  = df_combinations["cmpdname"].apply(literal_eval) #convert to list type
  df_combinations["CONCuM"]   = df_combinations["CONCuM"].apply(literal_eval)    #convert to list type

  df_combinations = df_combinations.explode(["cmpdname", "CONCuM"])

  df = pd.concat([df_combinations, df_singles], ignore_index=True, sort=False)

  #df

In [143]:
#@title (optional) change dose and compound names in PLAID?
change_concnames_PLAID         = True #@param {type:"boolean"}
change_cmpdnames_PLAID         = False #@param {type:"boolean"}

# e.g. assign dose from letter
if change_concnames_PLAID == True:
  #newconc = {"a": 0.001, "b": 0.01, "c": 0.1, "d": 1, "e": 10} # here you assign the new names of choice! 
  newconc = {"0.1%": 0, "X":3}
  df = df.replace({"CONCuM": newconc})

if change_cmpdnames_PLAID == True:
  newcmpd = {"1": "cmpx", "2": "cmpy", "3": "cmpz", "8": "cmpa", "9": "cmpb"} 
  df = df.replace({"cmpdname": newcmpd})

  #print a summary
  replicates = df.groupby(["plateID","well"])["cmpdname"].apply(lambda x: ','.join(x)).to_frame("cmpdname") # account for combinations!
  replicates = replicates.reset_index()
  replicates = replicates.groupby(["cmpdname"]).sum().reset_index()                  
  replicates = replicates[["cmpdname"]].value_counts().to_frame('counts')

#replicates

In [144]:
#@title Select units used in experiment (plaid)

unit                  = "uM" #@param ["pM","nM","uM","mM","dilution"]

# default is uM - here we will recalculate if you have put in another format
df['CONCuM'] = pd.to_numeric(df['CONCuM'], errors='coerce')

if unit == "uM":
  None
elif unit == "pM":
  df['CONCuM'] =  df["CONCuM"]/ 1000000
elif unit == "nM":
  df['CONCuM'] = df["CONCuM"]/ 1000
elif unit == "mM":
  df['CONCuM'] = df["CONCuM"] * 1000
elif unit == "dilution":
  df['CONCuM'] = df["CONCuM"] * 1000

  
# 1 Micromolar [µM] = 0.001 Millimolar [mM]
# 1 Micromolar [µM] = 1000 Nanomolar [nM]
# 1 Micromolar [µM] = 1.0×106 Picomolar [pM] (1000000)


In [145]:
#@title assign treatment groups

def treatmentstodict(conditions):
    DMSO, ctrls, blank, trt = ([] for i in range(4))
    catdict = {}
    
    for i in conditions:
        if bool(re.search('.*dmso.*', i,re.IGNORECASE)):
            findDMSO = re.findall('.*dmso.*', i, re.IGNORECASE) 
            DMSO.append(i)
            if findDMSO[0] not in catdict:
                catdict[i] = "DMSO" 
            

        elif bool(re.search('.*blank.*', i,re.IGNORECASE)):
            findblank = re.findall('.*blank.*', i, re.IGNORECASE) 
            blank.append(i)
            if findblank[0] not in catdict:
                catdict[i] = "blank" 
            

        elif bool(re.search(r'\[.*?\]', i)):
            findctrl = re.findall(r'\[[a-zA-Z0-9_]{4}?\]', i) 
            ctrls.append(i)
            if findctrl[0] not in catdict:
                catdict[i] = "ctrl" 
            
        else:
            if i not in catdict:
                trt.append(i)
                catdict[i] = "trt" 
    return catdict


conditions                 = np.unique(df[['cmpdname']].values).tolist()
catdict                    = treatmentstodict(conditions)
df["treatment_type"]       = df["cmpdname"].map(catdict)
df["treatment_type"].value_counts().to_frame('number of wells')

,number of wells
trt,1635
DMSO,260
ctrl,25


In [146]:
#@title Assign the highest available stock to each condition
maxmM_treat           = 10 #@param {type:"number"}
maxmM_ctrl            = 10 #@param {type:"number"}

# advanced settings: different stocks for each compound
advanced_manual_list_stocks           = False #@param {type:"boolean"}

In [147]:
#@title assign highest available stock
# and then we link it to your file
if advanced_manual_list_stocks == True: 
  file = next(iter(uploaded_file))
  manualstocks = pd.read_csv(file, usecols=["cmpdname","CONCuM","highest_stock_mM"])
  manualstocks['cmpd_dose'] = manualstocks['cmpdname'] + "[" +  manualstocks['CONCuM'].astype(str) + "]"
  manualstocks = manualstocks[["cmpd_dose","highest_stock_mM"]]
  manualstocksdict = dict(zip(manualstocks.cmpd_dose, manualstocks.highest_stock_mM))

  df['cmpd_dose']              = df['cmpdname'] + "[" +  df['CONCuM'].astype(str) + "]"
  df["highest_stock_mM"]       = df["cmpd_dose"].map(manualstocksdict)
  df = df.drop('cmpd_dose', axis=1) 
else:
  None

# if answered "No" manual list it will assign stocks based on answer above

def stockhighestmM(treatment_type,maxmM_treat,maxmM_ctrl):
    if treatment_type == "trt":
        return maxmM_treat
    if treatment_type == "ctrl":
        return maxmM_ctrl
    if treatment_type == "DMSO":
        return 0
    if treatment_type == "blank":
        return 0

if advanced_manual_list_stocks == False: 
  df["highest_stock_mM"]     = df.apply(lambda x: stockhighestmM(x['treatment_type'],maxmM_treat,maxmM_ctrl), axis=1)


In [148]:
#@title Then we find an optimal stock concentration for each well
dmso_percmax           = 1 #@param {type:"number"}

def log10range(max_mM):
    availstocks_mM = []  
    
    maxlog10 = int(math.log(max_mM,10))
    
    for i in range(-12, maxlog10 + 1):
        availstocks_mM.append(pow(10, i))
        
    return availstocks_mM
  
def stockfinder(concUM,highest_stock_mM,V2_ul,dmso_percmax): 
    
    if highest_stock_mM != 0:
        availstocks_mM = log10range(highest_stock_mM)

        if sourceplate_type == "S.200":
          MinV1_nl = 30 
        else:
          MinV1_nl = 8

        MaxV1_nl = (dmso_percmax / 100) * (V2_ul*1000)                                                                  

        C1_low  = (V2_ul * concUM) / MaxV1_nl                                                  
        C1_high = (V2_ul * concUM) / MinV1_nl                                                 

        psblstocks = [x for x in availstocks_mM if x >= C1_low and x <= C1_high]    
        
        if psblstocks:
            highestStock = max(psblstocks)   # select highest stock for your condition                                            
            return highestStock    
        else:
            raise Exception("not possible to find a suitable stock for requested settings")
    else:
        return 0

df["stock_conc_mM"]        = df.apply(lambda x: stockfinder(x['CONCuM'],x["highest_stock_mM"],V2_ul,dmso_percmax), axis=1)

In [149]:
#@title Calculate volumes for spotting

# calculate the volumes for spotting

def uLfromstock(concUM,stock_conc_mM,V2_ul):
    concUM =  (concUM * V2_ul) / stock_conc_mM if stock_conc_mM != 0 else 0
    return concUM / 1000

df["Volume [uL]"]          = df.apply(lambda x: uLfromstock(x["CONCuM"], x["stock_conc_mM"], V2_ul), axis=1)


# warn if volume is higher than plate max
highestvolume = df["Volume [uL]"].max()
print("highest volume in plate:",highestvolume, "ul")

wellcapacity = int(max_volume *1e06) # wellcapacity based on idot plate
if highestvolume > wellcapacity:
  raise Exception("The volume needed for some wells (highestvolume) exceeds the I.DOT well capacity (80ul), revise your setup !!") 

# we also reformat some labels to work with the idot 

def removeleadingzero(x):
    x = x[0] + x[1:3].lstrip("0")
    return x 
  
df["Target Well"]          = df["well"].apply(removeleadingzero)
df["Liquid Name"]          = df['cmpdname'] + "[" +  df['stock_conc_mM'].astype(str) + "]"
df.rename(columns          = {'plateID':'Target Plate'}, inplace = True)
df                         = df[["Target Plate","cmpdname","highest_stock_mM","stock_conc_mM","treatment_type","Target Well","Liquid Name","Volume [uL]"]]

highest volume in plate: 0.04 ul


In [150]:
#@title Normalize DMSO

DMSOstrat            = "To highest in plates" #@param ["None", "To highest in plates", "To fixed percentage"]
dmso_fixed_percmax   = 0.5 #@param {type:"number"}

In [151]:
#@title Generate a target plate for DMSO
# change max DMSO for selected strategy
# obs as of now limited to 96 wells 

grouped_df =           df.groupby(["Target Plate","Target Well"]).sum().reset_index() # account for combinations!

if DMSOstrat == "None":
  maxDMSO = 0
    
elif DMSOstrat == "To highest in plates":
  maxDMSO            = grouped_df.loc[grouped_df["Volume [uL]"].idxmax()]
  maxDMSO            = maxDMSO["Volume [uL]"]
    
elif DMSOstrat == "To fixed percentage":
  maxDMSOfrac    = dmso_fixed_percmax / 100
  maxDMSO        = maxDMSOfrac * (V2_ul) 



def normalizeDMSO(mydf):
    dfDMSO = grouped_df
    dfDMSO["DMSO_backfill_uL"] = maxDMSO - dfDMSO["Volume [uL]"]
    dfDMSO["DMSO_backfill_uL"][dfDMSO["DMSO_backfill_uL"] < 0] = 0
    dfDMSO                     = dfDMSO[dfDMSO.DMSO_backfill_uL != 0] 
    dfDMSO.drop(["Volume [uL]"], axis=1)
    dfDMSO["Volume [uL]"]      = dfDMSO["DMSO_backfill_uL"] 
    
    dfDMSO[["Liquid Name","cmpdname","treatment_type"]]   = "DMSO"
    dfDMSO[["highest_stock_mM","stock_conc_mM"]]          = 0
    dfDMSO = dfDMSO[["Target Plate","cmpdname","highest_stock_mM","stock_conc_mM","treatment_type","Target Well","Liquid Name","Volume [uL]"]]
    return dfDMSO


dfDMSO = normalizeDMSO(grouped_df)

print("DMSO in each well will be filled up to:", maxDMSO ,"ul DMSO, corresponding to: ", maxDMSO/V2_ul*100, "% DMSO")
print("A total of",len(dfDMSO), "wells are normalized")

DMSO in each well will be filled up to: 0.04 ul DMSO, corresponding to:  0.1 % DMSO
A total of 285 wells are normalized


In [152]:
dfDMSO

,Target Plate,cmpdname,highest_stock_mM,stock_conc_mM,treatment_type,Target Well,Liquid Name,Volume [uL]
2,plate_1,DMSO,0,0,DMSO,A11,DMSO,0.040
7,plate_1,DMSO,0,0,DMSO,A16,DMSO,0.040
16,plate_1,DMSO,0,0,DMSO,A24,DMSO,0.040
30,plate_1,DMSO,0,0,DMSO,B15,DMSO,0.028
37,plate_1,DMSO,0,0,DMSO,B21,DMSO,0.040
...,...,...,...,...,...,...,...,...
1879,plate_5,DMSO,0,0,DMSO,O16,DMSO,0.040
1888,plate_5,DMSO,0,0,DMSO,O24,DMSO,0.040
1898,plate_5,DMSO,0,0,DMSO,P11,DMSO,0.040
1902,plate_5,DMSO,0,0,DMSO,P15,DMSO,0.040


In [153]:
#@title Merge all target plates

frames      = [df,dfDMSO] 
target      = pd.concat(frames)
target      = target[target["Volume [uL]"] != 0]



# Now we will generate source plates

In [154]:
#@title Source plate preferences

sourceplate_strat            = 'automation' #@param ["automation", "simple"]
#seperate_source_plates       = False #@param {type:"boolean"}

In [155]:
#@title Generate DMSO source plate

def createplate(size, direction):

    import string

    if size == 96:
        colr = 13
        rowr = 8
    
    if size == 384:
        colr = 25
        rowr = 16
    

    row = list(string.ascii_uppercase[:rowr])
    col = [(f'{i:02d}') for i in range(1, colr, 1)]
    wells = []
      
    if direction == "vert":
        for c in col:
            for r in row:
                wells.append(str(r+c))
        return(wells)
    
    else:
        for r in row:
            for c in col:
                wells.append(str(r+c))
        return(wells)

def assign_DMSOsource(max_volume, dfvolumes):
    
    
    wellcapacity = int(max_volume *1e06) # wellcapacity based on idot plate
    well_state = {"well_number": 0, "current_amount": wellcapacity}
    #dfvolumes = dfvolumes.to_list()
    DMSOwells = createplate(size=96,direction="vert")
    
    sourcewelllist = []

    for volume in dfvolumes:
      #print(volume)
        remaining = well_state["current_amount"] - volume
        if remaining < 0:
            well_state["well_number"] += 1
            well_state["current_amount"] = wellcapacity
            wellindex = well_state["well_number"]
            sourcewelllist.append(DMSOwells[wellindex])
        else:
            well_state["current_amount"] -= volume
            wellindex = well_state["well_number"]
            sourcewelllist.append(DMSOwells[wellindex])
            
    sourcewells = [*set(sourcewelllist)]      
    return sourcewelllist

dfvolumes = dfDMSO["Volume [uL]"].to_list()
dfDMSO["Source Well"] = assign_DMSOsource(max_volume, dfvolumes)
dfDMSO[['Liquid Name','Source Plate',]] = 'DMSO', 'DMSOsource'
dmsoSOURCE = dfDMSO[["Liquid Name","Source Plate","Source Well"]].drop_duplicates()

In [156]:
#@title Visualize and save controls Source plates

if len(dmsoSOURCE.columns) == 0:
  print('no controls source plate was created')
else:
  DMSOlist = dmsoSOURCE['Source Plate'].unique().tolist()
  for plate in DMSOlist:
      df = dmsoSOURCE[dmsoSOURCE['Source Plate'] == plate ] 
      df['Metadata_Column'] =    df['Source Well'].astype(str).str[1:3]
      df['Metadata_Row']    =    df['Source Well'].astype(str).str[0]
      DMSOpivot = df.pivot(columns="Metadata_Column", index="Metadata_Row", values="Liquid Name") 
      print("controls source plate:") 
      DMSOpivot 

      DMSOpivot.to_csv("{}_{}.csv".format(protocol_name,plate), encoding = 'utf-8-sig') 
      files.download("{}_{}.csv".format(protocol_name,plate))

DMSOpivot

controls source plate:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Metadata_Column,01
Metadata_Row,
A,DMSO


# source plates for compounds

In [157]:
df2 = target.copy()

In [158]:
#@title Generate source plates for treatment and controls (option 2: automation friendly)
def assignsource(df2,treatment_type):
  
    if len(df2[df2["treatment_type"]==str(treatment_type)]) == 0:
        print("OBS: no", treatment_type, "found")

    else:
        print(treatment_type)
        seldf                     = df2[df2["treatment_type"]==str(treatment_type)] # make subset of treatment type
        seldf["stock_conc_mM"]    = seldf.stock_conc_mM.astype(float)
        seldf["dilutions1in10"]   = np.log10(seldf["highest_stock_mM"].astype(int) / seldf["stock_conc_mM"]).astype(int) + 1 # how many dilution steps are needed
        seldf                     = seldf.groupby(["Liquid Name"])[["cmpdname","stock_conc_mM","dilutions1in10","treatment_type"]].max().reset_index()
        maxdils     = seldf["dilutions1in10"].max() # max number of dilutions in this treatment type
        print("max number of dilutions in treatment group:",maxdils)
        compounds   = np.unique(seldf[["cmpdname"]].values).tolist()
        compounds   = sorted(compounds)
        nrcompounds = len(compounds)
        print("total number of compounds:", nrcompounds)


        # ----------------- source plates ------------------#
        nrsubplates        = math.floor(12 / maxdils)    # calculate how many subplates per 96 well source plate
        maxcmpdperplate    = nrsubplates * 8             # how many compounds do fit on one 96 well plate?
        totalsubplates     = math.ceil(nrcompounds / 8)  # total number of subplates
        print("total number of subplates:", totalsubplates)
        print("number of subplates per 96 well plate:", nrsubplates)
        plates             = math.ceil(nrcompounds / maxcmpdperplate)  # total number of source plates
        plates             = int(plates)
        print("total number of source plates:", plates)

        # --------------- create plates and subplates------#
        x         = int(math.floor(12/ nrsubplates)) # start position each subplate
        startcol  = [int(f"{i:01d}") for i in range(1, 13, x)] * plates
        startcols = startcol[0:totalsubplates]
        row96     = list(string.ascii_uppercase[:8])
        welldict = {}
        #print(startcols)

        for i, cmp in enumerate(compounds):
            for j, dilution in enumerate(range(0,maxdils)):
                comp = i+1                                                  # counts chemicals starting at 1
                subplate = math.floor(comp/8)                               # decides on subplate (e.g. compound 20 will be on 3th subplate)
                subplateindex = subplate - 1                                # starts counting subplates at zero
                plate = math.floor(subplate / nrsubplates)                  # source plate number
                plate  = treatment_type + "_" + str(plate + 1)              # name of subplate

                # assigns row letter for compound
                #cmprow = str(cmprow[i])   
                cmprow = (list(string.ascii_uppercase[:8]) * totalsubplates)[0:len(compounds)] # generated list of letters (A-H) for the number of subplates 
                cmprow = str(cmprow[i]) 
                #print(cmprow) 
                                                                
                # assign column number for concentration
                cmpcol = startcols[subplate]
                cmpcol = cmpcol + j
                cmpcol = str(cmpcol).zfill(2)
                #print(cmpcol)

                # configures well location
                well = cmprow+cmpcol
                #print(well)

                # dictionary of compound, well, dilution and platename
                welldict[i,j] = [well,cmp,j+1,plate]

        source      = pd.DataFrame.from_dict(welldict,orient="index", columns=["Source Well","cmpdname","dilutions1in10", "Source Plate"])
        sourceplate = pd.merge(seldf, source,  how="left", left_on=["cmpdname","dilutions1in10"], right_on = ["cmpdname","dilutions1in10"])
        return(sourceplate)
if sourceplate_strat=="automation":
  trtSOURCE  = assignsource(df2,"trt")
  ctrlSOURCE  = assignsource(df2,"ctrl")

trt
max number of dilutions in treatment group: 1
total number of compounds: 981
total number of subplates: 123
number of subplates per 96 well plate: 12
total number of source plates: 11
ctrl
max number of dilutions in treatment group: 1
total number of compounds: 5
total number of subplates: 1
number of subplates per 96 well plate: 12
total number of source plates: 1


In [165]:
trtSOURCE

,Liquid Name,cmpdname,stock_conc_mM,dilutions1in10,treatment_type,Source Well,Source Plate
0,100[10],100,10.0,1,trt,C01,trt_1
1,101[10],101,10.0,1,trt,D01,trt_1
2,102[10],102,10.0,1,trt,E01,trt_1
3,103[10],103,10.0,1,trt,F01,trt_1
4,104[10],104,10.0,1,trt,G01,trt_1
...,...,...,...,...,...,...,...
976,980[10],980,10.0,1,trt,C03,trt_11
977,981[10],981,10.0,1,trt,D03,trt_11
978,98[10],98,10.0,1,trt,B03,trt_11
979,99[10],99,10.0,1,trt,E03,trt_11


In [159]:
#@title Visualize and save treatment Source plates

if len(trtSOURCE.columns) == 0:
  print('no controls source plate was created')
else:
  trtlist = trtSOURCE['Source Plate'].unique().tolist()
  for plate in trtlist:
      df = trtSOURCE[trtSOURCE['Source Plate'] == plate ] 
      df['Metadata_Column'] =    df['Source Well'].astype(str).str[1:3]
      df['Metadata_Row']    =    df['Source Well'].astype(str).str[0]
      trtpivot = df.pivot(columns="Metadata_Column", index="Metadata_Row", values="Liquid Name") 
      print("treatment source plate:") 
      #print(pivot) 

      trtpivot.to_csv("{}_{}.csv".format(protocol_name,plate), encoding = 'utf-8-sig') 
      files.download("{}_{}.csv".format(protocol_name,plate))

trtpivot

treatment source plate:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

treatment source plate:


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

treatment source plate:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Metadata_Column,01,02,03
Metadata_Row,,,
A,964[10],971[10],979[10]
B,965[10],972[10],98[10]
C,966[10],973[10],980[10]
D,967[10],974[10],981[10]
E,968[10],975[10],99[10]
F,969[10],976[10],NaN
G,97[10],977[10],NaN
H,963[10],970[10],978[10]


In [160]:
#@title Visualize and save controls Source plates

if len(ctrlSOURCE.columns) == 0:
  print('no controls source plate was created')
else:
  ctrllist = ctrlSOURCE['Source Plate'].unique().tolist()
  for plate in ctrllist:
      df = ctrlSOURCE[ctrlSOURCE['Source Plate'] == plate ] 
      df['Metadata_Column'] =    df['Source Well'].astype(str).str[1:3]
      df['Metadata_Row']    =    df['Source Well'].astype(str).str[0]
      ctrlpivot = df.pivot(columns="Metadata_Column", index="Metadata_Row", values="Liquid Name") 
      print("controls source plate:") 
      ctrlpivot 

      ctrlpivot.to_csv("{}_{}.csv".format(protocol_name,plate), encoding = 'utf-8-sig') 
      files.download("{}_{}.csv".format(protocol_name,plate))

ctrlpivot

controls source plate:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Metadata_Column,01
Metadata_Row,
A,[etop][10]
B,[fenb][10]
C,[meto][10]
D,[tetr][10]
E,[berb][10]


In [161]:
#@title merge all source plates

frames = [trtSOURCE, ctrlSOURCE, dmsoSOURCE]
source = pd.concat(frames)
source['Source Well']          = source['Source Well'].apply(removeleadingzero)

In [162]:
#@title Next step is to create and format the idot protocol

sourcewell = source[["Liquid Name","Source Well"]]
sourcedictwell = dict(sourcewell.values)

sourceplate = source[["Liquid Name","Source Plate"]]
sourceplatedict = dict(sourceplate.values)

target["Source Well"]  = target["Liquid Name"].map(sourcedictwell)
target["Source Plate"] = target["Liquid Name"].map(sourceplatedict)

sourceplates = source['Source Plate'].unique().tolist()
targetplates = target['Target Plate'].unique().tolist()

sourceplates = sourceplates[::-1]

targetformat = target[["Target Plate","Target Well","Volume [uL]","Liquid Name"]]

collected_df = []
i = 1

for splate in sourceplates:
    for tplate in targetplates:
        df = target.loc[((target["Source Plate"] == splate) & (target["Target Plate"] == tplate ) )]
                      
        df = df[["Source Well","Target Well","Volume [uL]","Liquid Name"]]
        df = df.reindex(columns=[*df.columns.tolist(), "", "","",""], fill_value="")
          
        df = pd.concat([df.columns.to_frame().T, df], ignore_index=True)
        df.columns = range(len(df.columns)) 
        
        subheader = [[sourceplate_type, splate, "",max_volume, target_plate_type, tplate, "",waste],
        ["DispenseToWaste="+str(dispense_to_waste),"DispenseToWasteCycles="+str(dispense_to_waste_cycles),"DispenseToWasteVolume="+str(dispense_to_waste_volume),"UseDeionisation="+str(use_deionisation),"OptimizationLevel="+str(optimization_level),"WasteErrorHandlingLevel="+str(waste_error_handling_level),"SaveLiquids="+str(save_liquids),""]]
        subheader = pd.DataFrame(subheader)
        
        protocol = pd.concat([subheader, df],ignore_index=True)
        collected_df.append(protocol)

header = [[protocol_name, software, user_name, date, time,"","",""]]
header = pd.DataFrame(header)

dfs = pd.concat(collected_df)
fullprotocol = pd.concat([header, dfs],ignore_index=True)


In [163]:
#@title Save the I.DOT protocol

fullprotocol.to_csv("IDOT_{}.csv".format(protocol_name), header=False, index=False,encoding = 'utf-8-sig') 
files.download("IDOT_{}.csv".format(protocol_name))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [164]:
#@title see protocol
fullprotocol

,0,1,2,3,4,5,6,7
0,RMS_SPECS,1.7.2021.1019,Jonne,01/26/23,21:27:15,,,
1,S.100 Plate,DMSOsource,,0.00008,MWP 384,plate_1,,Waste Tube
2,DispenseToWaste=True,DispenseToWasteCycles=3,DispenseToWasteVolume=1e-07,UseDeionisation=True,OptimizationLevel=ReorderAndParallel,WasteErrorHandlingLevel=Ask,SaveLiquids=Ask,
3,Source Well,Target Well,Volume [uL],Liquid Name,,,,
4,A1,A11,0.04,DMSO,,,,
...,...,...,...,...,...,...,...,...
2136,DispenseToWaste=True,DispenseToWasteCycles=3,DispenseToWasteVolume=1e-07,UseDeionisation=True,OptimizationLevel=ReorderAndParallel,WasteErrorHandlingLevel=Ask,SaveLiquids=Ask,
2137,Source Well,Target Well,Volume [uL],Liquid Name,,,,
2138,S.100 Plate,trt_1,,0.00008,MWP 384,plate_5,,Waste Tube
2139,DispenseToWaste=True,DispenseToWasteCycles=3,DispenseToWasteVolume=1e-07,UseDeionisation=True,OptimizationLevel=ReorderAndParallel,WasteErrorHandlingLevel=Ask,SaveLiquids=Ask,
